# Tweet Scrapping and extraction
This is the first note book where I did the extraction of the tweets used  for the project, in the second notebook, I did a detailed exploratory data analysis and sentiment analysis, yoou can check it here.
I also wrote a story-like report report which you can check out on medium.

In [ ]:

!pip install twitterscraper

     |████████████████████████████████| 92kB 3.8MB/s 
  Created wheel for twitterscraper: filename=twitterscraper-1.6.1-cp37-none-any.whl size=11347 sha256=82023622e5e85b266bca072d49bb992d9d61d82e54afbb217851df0365bb10ab
  Stored in directory: /root/.cache/pip/wheels/1a/50/a0/808fd6df0ac14c168b73874056a1ca52e4d88c0c1ecedd4e48
Successfully built twitterscraper


In [ ]:
#To read the uploaded file that contains the key we need configparser
!pip install ConfigParser

So I already uploaded my api and access keys and scretes to my drive in a .txt file named tweeter.api.properties, it could be anyname though.

In [1]:
import configparser
config = configparser.RawConfigParser()
config.read(filenames = '/content/drive/MyDrive/Colab Notebooks/Twitterapi.properties.txt')
#getting all hthe sections in the file
print(config.sections())

['twitter']


In [4]:
#import the neccessary libraries
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import pandas as pd
import os
import time

In [2]:
#accessing API 
accesstoken = config.get('twitter', 'accesstoken') # config.get takes the aurgement 'section' and  'the line title'
accesstokensecrete = config.get('twitter', 'accesstokensecret')
apikey = config.get('twitter', 'apikey')
apikeysecret = config.get('twitter', 'apikeysecret')

### For the first Exctration method, I will be calling the twitter authenticator directly

In [5]:

auth = OAuthHandler(apikey, apikeysecret)
auth.set_access_token(accesstoken, accesstokensecrete)
api = tweepy.API(auth)


In [7]:
#creating a function that does all the scrapping and coverts to a csv file
def scraptweets(search_words, date_since, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the data:
    global  db_tweets
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags'])
    program_start = time.time()
    for i in range(0, numRuns):
      # We will time how long it takes to scrape tweets for each run:
      start_run = time.time()

      # Collect tweets using the Cursor object
      # .Cursor() returns an object that you can iterate or loop over to access the data collected.
      # Each item in the iterator has various attributes that you can access to get information about each tweet
      
      tweets = tweepy.Cursor(api.search, q=search_words, lang="en", since=date_since, tweet_mode='extended').items(numTweets)
      
      # Store these tweets into a python list
      tweet_list = [tweet for tweet in tweets]
      
      
      # Obtain the following info (methods to call them out):
      # user.screen_name - twitter handle
      # user.description - description of account
      # user.location - where is he tweeting from
      # user.friends_count - no. of other users that user is following (following)
      # user.followers_count - no. of other users who are following this user (followers)
      # user.statuses_count - total tweets by user
      # user.created_at - when the user account was created
      # created_at - when the tweet was created
      # retweet_count - no. of retweets
      # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
      # retweeted_status.full_text - full text of the tweet
      # tweet.entities['hashtags'] - hashtags in the tweet

      # Begin scraping the tweets individually:
      noTweets = 0
      for tweet in tweet_list:
        # Pull the values
        username = tweet.user.screen_name
        acctdesc = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        totaltweets = tweet.user.statuses_count
        usercreatedts = tweet.user.created_at
        tweetcreatedts = tweet.created_at
        retweetcount = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
        try:
            text = tweet.retweeted_status.full_text
        except AttributeError:  # Not a Retweet
            text = tweet.full_text
        # Add the 11 variables to the empty list - ith_tweet:
        ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]
        # Append to dataframe - db_tweets
        db_tweets.loc[len(db_tweets)] = ith_tweet
        # increase counter - noTweets  
        noTweets += 1
        
      # Run ended:
      end_run = time.time()
      duration_run = round((end_run-start_run)/60, 2)
      
      print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
      print('time take for {} run to complete is {} mins'.format(i+1, duration_run))
          
      if i < numRuns:
        time.sleep(920) #15 minute sleep time
    # Once all runs have completed, save them to a single csv file:
    from datetime import datetime

    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    # Define working path and filename
    path = os.getcwd()
    filename = '/content/drive/MyDrive/Colab Notebooks/' + to_csv_timestamp + search_words + '_ogbeni_tweets.csv'
    # Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)

    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [ ]:
# Initialise these variables:
search_words = "@ogbenidipo OR Dr Dipo OR Ogbeni Dipo"
date_since = "2021-03-23"
numTweets = 3200
numRuns = 3

In [ ]:

# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 786
time take for 1 run to complete is 0.35 mins
no. of tweets scraped for run 2 is 786
time take for 2 run to complete is 0.35 mins
no. of tweets scraped for run 3 is 786
time take for 3 run to complete is 0.37 mins
Scraping has completed!
Total time taken to scrap is 47.083333333333336 minutes.


###Second Extration method by calling the authenticator via tweepy

In [26]:
auth = tweepy.OAuthHandler(apikey,apikeysecret) 
auth.set_access_token(accesstoken,accesstokensecrete) 
api = tweepy.API(auth,wait_on_rate_limit=True)
search_word = '@ogbenidipo OR Dr Dipo OR Ogbeni Dipo' 
date_since = '2021-03-31' # tweepy starts extraction from a day after the date-since
tweets = tweepy.Cursor(api.search,q = search_word, lang ='en',since = date_since).items(2000)
tweet_details = [[tweet.user.screen_name, tweet.user.location, tweet.favorite_count, tweet.user.followers_count, tweet.user.statuses_count,
                         tweet.user.created_at, tweet.created_at, tweet.retweet_count, tweet.text]for tweet in tweets]

In [25]:
print (len(tweet_details))
complex_tweets = pd.DataFrame(tweet_details, columns = ['username', 'location', 'likes',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text'])

641


In [18]:
complex_tweets.to_csv('/content/drive/MyDrive/Colab Notebooks/'+ date_since + search_word, index= False)

In [19]:
complex_tweets= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/' + date_since + search_word, parse_dates=True)

In [20]:
## a sample of what the dataframw looks like
complex_tweets.tail()


,username,location,likes,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text
637,fuski_,Portharcourt,0,156,2882,2013-01-15 10:11:32,2021-03-31 08:32:09,0,@OgbeniDipo Oga Dipo still dey beg oo...e say ...
638,fuski_,Portharcourt,76,156,2882,2013-01-15 10:11:32,2021-03-31 08:28:13,0,@OgbeniDipo Oga Dipo say all those young guys ...
639,PrinceFromDilli,🌐,0,204,7747,2013-03-26 13:10:22,2021-03-31 05:39:48,1,@guardian I don't know what this guy Dr dipo i...
640,ibjnr21,NaN,0,131,3981,2014-07-05 20:47:34,2021-03-31 04:49:42,0,"@OgbeniDipo Gbam Mr Dipo, you have said it all"
641,oz_enenia,Enugu,0,185,1241,2012-06-17 12:09:54,2021-03-31 00:07:39,0,@brainbizinfo @OgbeniDipo @akinalabi Will do. ...


#Continue in the Second notebook here